In [1]:
import numpy as np
from src.darwin_core_aggregations import DarwinCoreAggregations
from src.geojson import build_geojson_feature_collection
from run import parse_arguments
import numpy as np
from contexttimer import Timer
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import pdist
from src.cli import parse_arguments
from src.cluster_color_builder import ClusterColorBuilder
from src.cluster_stats import Stats
from src.darwin_core_aggregations import DarwinCoreAggregations
from src.render import plot_clusters
from src.cluster import ClusterId
import matplotlib.pyplot as plt
from src.geojson import build_geojson_feature_collection
import os
import logging
from sklearn.preprocessing import StandardScaler
from run import cluster, build_X

In [2]:
# params
input_file = "data-continental-united-states.csv"
geohash_precision = 3
show_dendrogram = True
log_file = "log.txt"
output_file = "output.geojson"

In [3]:
# set up logging
logging.basicConfig(filename=log_file, encoding="utf-8", level=logging.INFO)

In [4]:
darwin_core_aggregations = DarwinCoreAggregations.build(
    input_file, geohash_precision
)

In [ ]:
darwin_core_aggregations.unfiltered_taxon_counts

In [ ]:
darwin_core_aggregations.taxon_counts

In [ ]:
X = build_X(darwin_core_aggregations)
X.shape

In [ ]:

num_clusters = 15

cluster_dataframe = cluster(
    darwin_core_aggregations,
    num_clusters,
    show_dendrogram,
)

In [ ]:


# Find the top averages of taxon
all_stats = Stats.build(darwin_core_aggregations)

feature_collection = build_geojson_feature_collection(
    (
        cluster,
        geohashes,
        cluster_dataframe.determine_color_for_cluster(
            cluster, darwin_core_aggregations
        ),
    )
    for cluster, geohashes in cluster_dataframe.iter_clusters_and_geohashes()
)

# print_results(darwin_core_aggregations, all_stats, cluster_dataframe)

plot_clusters(feature_collection, num_clusters=num_clusters)

# write_geojson(feature_collection, args.output_file)

In [ ]:
from run import print_cluster_stats
cluster_id = 1
geohashes = cluster_dataframe.geohashes_for_cluster(cluster_id)
print_cluster_stats(cluster_id, geohashes, darwin_core_aggregations, all_stats)


In [ ]:

stats_for_clusters = {
    cluster: Stats.build(
        darwin_core_aggregations,
        geohash_filter=geohashes,
    )
    for cluster, geohashes in cluster_dataframe.iter_clusters_and_geohashes()
}
stats_for_clusters


In [ ]:
[
    {
        "cluster": cluster,
        "waterfowl": stats.waterfowl_count(),
        "aves": stats.aves_count(),
        "ratio": stats.waterfowl_count() / stats.aves_count(),
    }
    for cluster, stats in stats_for_clusters.items()
]

